In [1]:
# [credits.csv]
# movie_id : 영화의 고유 ID입니다. 이 ID는 영화를 공유하는 데 사용됩니다.
# title : 영화의 제목입니다.
# cast : 영화에 출연한 배우들의 정보가 JSON 형식으로 저장되어 있습니다. 각 배우의 정보에는 배터리 이름, 배우의 이름, 연결 등이 포함되어 있습니다.
# crew : 영화 제작에 나눔의 정보가 JSON 형식으로 저장되어 있습니다. 수확의 정보에는 직책(예: 감독, 작곡가 등), 수확의 이름 등이 포함되어 있습니다.

# [movies.csv]
# budget: 영화의 예산.
# genres: 영화와 관련된 장르 목록입니다.
# homepage: 영화의 홈페이지 URL입니다.
# id: 영화의 ID입니다.
# keywords: 영화와 관련된 키워드 목록입니다.
# original_language: 영화의 원래 언어입니다.
# original_title: 영화의 원래 제목입니다.
# overview: 영화에 대한 간략한 요약.
# popularity: 영화의 인기 점수.
# production_companies: 영화에 관련된 제작사 목록입니다.
# production_countries: 영화가 제작된 국가 목록입니다.
# release_date: 영화 개봉일입니다.
# revenue: 영화로 발생한 수익입니다.
# runtime: 영화의 실행 시간(분)입니다.
# spoken_languages: 영화에서 사용되는 언어 목록입니다.
# status: 영화의 현재 상태.
# tagline: 영화의 태그라인.
# title: 영화의 제목입니다.
# vote_average: 영화의 평균 득표율입니다.
# vote_count: 영화에 대한 투표 수입니다.


In [2]:
import pandas as pd
import numpy as np
import pickle
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer # Tf - id 단어들의 퓨처 벡터화
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df_credits = pd.read_csv('tmdb_5000_credits.csv') #영화 배우,감독?
df_movies = pd.read_csv('tmdb_5000_movies.csv') #영화 정보

C = df_movies['vote_average'].mean() #영화 평점의 평균값
m = df_movies['vote_count'].quantile(0.9) # 투표수가 적은 영화 10%

# 아이템의 인기도와 품질을 모두 고려한 가중평점을 구할수있어서 보다 균형잡힌 방식으로 아이템의 순위를 맥이고 우선순위를 정할수있다. 
def weighted_rating(x, m=m, C=C):
    v = x['vote_count'] #투표 횟수
    R = x['vote_average'] #평점의 평균값
    return (v / (v + m) * R) + (m / (m + v) * C)

def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

def get_list(x):
    if isinstance(x, list): # 타입변경 유효성검사
        names = [i['name'] for i in x]
        if len(names) > 3:
            names = names[:3]
        return names
    return []

def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(' ', '')) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(' ', ''))
        else:
            return ''
        
def overview_list(x): 
    return x.split()

def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres']) 

def total_soup(x):
    return ' '.join(x['overview']) + ' ' +' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres']) 

In [4]:
df_credits['title'].equals(df_movies['title'])
df_credits.columns = ['id', 'title', 'cast', 'crew']
df_movies = df_movies.merge(df_credits[['id', 'cast', 'crew']],on='id')

df_movies['overview'] = df_movies['overview'].fillna('') # 결측치 채우기

tfidf = TfidfVectorizer(stop_words='english') # 객체생성
tfidf_movies_matrix = tfidf.fit_transform(df_movies['overview']) # 벡터화 이후 학습
overView = cosine_similarity(tfidf_movies_matrix, tfidf_movies_matrix) # 줄거리 유사성계산

df_movies['overview'] = df_movies["overview"].str.replace(pat=r'[^\w]', repl=r' ', regex=True)
df_movies['overview'] = df_movies['overview'].apply(overview_list)

In [5]:

features = ['cast', 'crew', 'keywords', 'genres'] # 출연배우, 스텝, 영화 hash tag, 장르
for feature in features:
    df_movies[feature] = df_movies[feature].apply(literal_eval)

df_movies['director'] = df_movies['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    df_movies[feature] = df_movies[feature].apply(get_list)

features = ['cast', 'keywords', 'director', 'genres', 'overview']
for feature in features:
    df_movies[feature] = df_movies[feature].apply(clean_data)

df_movies['soup'] = df_movies.apply(create_soup, axis=1)
df_movies = df_movies.reset_index()
find_list = df_movies[['id', 'title']].copy()

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df_movies['soup'])
credits = cosine_similarity(count_matrix, count_matrix)

df_movies['total'] = df_movies.apply(total_soup, axis=1)
total_matrix = tfidf.fit_transform(df_movies['total'])
total = cosine_similarity(total_matrix, total_matrix)

In [6]:
pickle.dump(find_list, open('find_list.pickle', 'wb'))
pickle.dump(overView, open('overView.pickle', 'wb'))
pickle.dump(credits, open('credits.pickle', 'wb'))
pickle.dump(total, open('total.pickle', 'wb'))

In [7]:
df_movies_copy = df_movies.copy().loc[df_movies['vote_count'] >= m]
df_movies_copy['score'] = df_movies_copy.apply(weighted_rating, axis=1)
df_movies_copy = df_movies_copy.sort_values('score', ascending=False) # 가중치 내림차순